# **Lets create the big dataframe for blood**


**strucutre of this notebook :**
1. **Blood sites on blood data**
    - 1.1 imports 
    - 1.2 get all mutant sites 
    - 1.3 append non-mut sites --> no overlap with mutant sites (takes 20mins) 
    - 1.4 basic declarations for the loop --> nte that still not using all tracks (ie hapmap ) 
    - 1.5 diagnostics (Xed out for script: was an interactive chunk) 
    - 1.6 big daddy loop 
    - 1.7 loop print error lists 

## **1.1 imports** 

In [1]:
import random
import pandas as pd
import numpy as np 
from numpy.random import choice
import matplotlib.pyplot as plt
import collections
from Bio import AlignIO
import pysam 
from tqdm import tqdm 
from datetime import datetime
import gzip

## **1.2  mutant sites list** 

In [2]:
mutations_lines = open('../../data/germline/mutation_data/mutations_hg18_tabDelim.bed').readlines()

In [3]:
#create a dictionary where each chrom key will have a n empty list 
duplicate_lines = []
muts_bychrom_dict = {}
for x in range(1,23): 
    key_string = 'chr{n}'.format(n=x)
    muts_bychrom_dict[key_string] = []
    
#fill each chromosome's empty list  with the sites for that chrom 
non_chrnMuts = []#create list of chrom names that dont belong to chrN format --> disgnostic 
for line in mutations_lines[1:]: 
    if line[0]=="c":                                               #aking sure the line is a chrN (lots of weird junk..) 
        chrom_mut = line.split("\t")[0]
        mut_startSite = line.split("\t")[1]                  #getting rid of the weird double(hgopefully) 
        if chrom_mut in muts_bychrom_dict.keys():                  #controlling for chrX/chrY
            if mut_startSite not in muts_bychrom_dict[chrom_mut]: 
                muts_bychrom_dict[chrom_mut].append(mut_startSite)
            else: duplicate_lines.append(line)
        else: 
            non_chrnMuts.append(chrom_mut)

        
#testing making usre the only sites that dont make it are sex chromosome mutations 
print(len(non_chrnMuts),"  non chrN muts (ommited) from these lables:", np.unique(non_chrnMuts))

#add the sites infro from file 
sites = []#sites = list of sites 
for chrom_key in muts_bychrom_dict.keys(): 
    for mutation_element in muts_bychrom_dict[chrom_key]: 
        
        sites.append([chrom_key, int(mutation_element),1]) #the 1 is for mutation status column. 1 = yes 
        
print("number included mutations = " +str(len(sites)))

62   non chrN muts (ommited) from these lables: ['chr13_random' 'chr17_random' 'chr19_random' 'chr21_random'
 'chr22_random' 'chr4_random' 'chr6_random' 'chr9_random' 'chrX']
number included mutations = 104941


## **1.3 append non-mutations to sites list** 

In [4]:
#get chrom length information so I can perform weighted choice for non-mut site selection 
ChromLengths = pd.read_csv('../../data/global/sequence/hg38_chromosomelengths.csv') #read in the csv file of hg38 chrom lengths I found on the internets 
total_length=0 #lets sum (get the total length) 
for length in list(ChromLengths.Length): 
    total_length+=int(length.replace(",",""))

#build dictionary to store porbability 
dict_lengths = {}#creat emepty dictionary 
for x in range (0,22): 
        tmp_index = x +1
        length = str(ChromLengths[x:x+1]).split()[4]
        length = length.replace(",", "")
        length = int(length)
        dict_lengths["chr"+str(tmp_index)] = length

#make the porbability of choosing a chrom based on length 
list_0chroms = ['chr01', 'chr02', 'chr03','chr04','chr05','chr06','chr07','chr08','chr09','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chr22']
list_chroms = ['chr' + str(i) for i in range(1, 23)]
list_chrom_probabilities = []
for chrom in list_chroms: 
    list_chrom_probabilities.append(dict_lengths[chrom]/total_length)

#maing sure the probability list sums to one (arbitrarily add the rounding error to chr1)
list_chrom_probabilities[0] = list_chrom_probabilities[0]+1-sum(list_chrom_probabilities) # adds the 0.00000001 left from rounding errors to the chr1 so sum adds perfectly to 1. 
assert(sum(list_chrom_probabilities)==1)

#perfrom the non-mutant site draw 
number_nonmuts = len(sites)
chrom_draw = choice(list_chroms, number_nonmuts,p=list_chrom_probabilities)

#visualize the draw: 
#print(collections.Counter(draw))

#make the sites list with the chr# and site 
for chrom in tqdm(chrom_draw): 
    site_draw = random.randrange(1,dict_lengths[chrom])
    if [chrom,site_draw] not in sites: 
        sites.append([chrom,site_draw,0]) # the 0 if for the mutation status column. 0 = no 
# print(len(sites))
#haha there was one mutation overlap!! 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 104941/104941 [03:00<00:00, 580.74it/s]


## **1.4 basic declarations for the loop** 

**tissue specific declarations (may chnage b/t somatic models )** 

In [5]:
mutations_df = pd.read_table('../../data/germline/mutation_data/mutations_hg18_tabDelim.bed', header = None)

In [6]:
#rename header for easier access down the line 
#replace space with underscore, / with Or, and Age(yars) with AgeInYears
mutations_df.columns = ["chrom","pos","fake_end","ref","alt","Fathers_age_at_conception","Mothers_age_at_conception"]#this has to chnage if i modify the mutation donwload script 

mutations_list = mutations_df.pos.values.tolist()  # may need to chnage the name ofwhich coloumn has the mutations 

In [7]:
mutations_df.head()

,chrom,pos,fake_end,ref,alt,Fathers_age_at_conception,Mothers_age_at_conception
0,chr1,544956,544957,C,T,39,37
1,chr1,712003,712004,C,G,36,30
2,chr1,769193,769194,A,G,29,32
3,chr1,814664,814665,T,G,34,31
4,chr1,830494,830495,A,G,32,21


**declarations that should stay the same for all tissues** 

In [8]:
#dictionry where i specify which col contains the information in the datafile , 0 indexed 
tracksColFile_dict = {"H3k27_female":[3,"specific","Na=0","../../data/germline/track_data/H3k27/H3k27ac_female_hg18_sorted.bed.gz"], 
                      "H3k27_male":[3,"specific","Na=0","../../data/germline/track_data/H3k27/H3k27ac_male_hg18_sorted.bed.gz"],
                      "H3k4me1_female":[3,"specific","Na=0","../../data/germline/track_data/H3k4me1/H3k4me1_female_hg18_sorted.bed.gz"], 
                      "H3k4me1_male":[3,"specific","Na=0","../../data/germline/track_data/H3k4me1/H3k4me1_male_hg18_sorted.bed.gz"], 
                       "H3k4me3_female":[3,"specific","Na=0","../../data/germline/track_data/H3k4me3/H3k4me3_female_hg18_sorted.bed.gz"], 
                       "H3k4me3_male":[3,"specific","Na=0","../../data/germline/track_data/H3k4me3/H3k4me3_male_hg18_sorted.bed.gz"], 
                      "DNAse_female":[3,"specific","Na=0","../../data/germline/track_data/DNAse/DNAse_female_hg18_sorted.bed.gz"], 
                      "DNAse_male":[3,"specific","Na=0","../../data/germline/track_data/DNAse/DNAse_male_hg18_sorted.bed.gz"], 
                      "Transcription_male":[3,"specific","Na=0","../../data/germline/track_data/transcription/transcription_male_hg18_sorted.bed.gz"],
                      "Transcription_female":[3,"specific","Na=0","../../data/germline/track_data/transcription/transcription_female_hg18_sorted.bed.gz"],
                                            
                      
                      "laminB1":[4,"global","Na=0","../../data/global/track_data/laminB1/hg18.laminB1.bed.gz"], 
                      "recombination": [4,"global","Na=Na","../../data/global/track_data/recombination/recombination.bed.gz"],
                       "Repeats":["binary","global","Na=0", "../../data/global/track_data/repeats/repeats.bed.gz"],
                      "Replication":[3,"global","Na=0","../../data/global/track_data/replication/replication.bed.gz"]} #remove nucmec/dennis, phastcons, 

In [9]:
#basic declarations (constants) 

fastas_dict = {}   # creating dictionary with fasta alignment, length of seq, 
for chrom in tqdm(list_chroms):
    filename_tmp = "../../data/global/sequence/{c}.fa.gz".format(c=chrom)
    fastas_dict[chrom] = []
    with gzip.open(filename_tmp, "rt") as handle:
        fastas_dict[chrom].append(AlignIO.read(handle,"fasta"))
        alignment_tmp = fastas_dict[chrom][0]
        fastas_dict[chrom].append(len(str(alignment_tmp[0].seq)))


#variable declaration  
list_of_surrounding_contexts = [0,100,10000] #note if you chnagee/ increase this, then you oncrese the buffer zone (not using sites in the buffer, len(dna)-max_distance )
distance_max = max(list_of_surrounding_contexts)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:53<00:00,  2.41s/it]


# **1.6 big daddy loop** 

**table creation notes** 
1. no overlapping non-muts sites 
2. sites are picked allover the genome (ind of buffer) but then sites outside buffer excluded from table 
    - can keep track of sites in "list_sites_outOfBufferrange
3. if the site was mutant, then triplet centre base chnages (not good!) 

In [ ]:
#nned to be reset every simulation 
alignment_discordant = []                                    #used to collect all the instances where the hg18 fasta result doesnt match the mutation dataset "Ref" column 
list_no_seq_at_site = []
list_sites_outOfBufferRange = []

#shorter verison for now 
sites = sites[int(len(sites)/4):int(len(sites)/2)]

#code: 
timestamp = datetime.now().strftime("%Y/%m/%d").replace("/", "_").replace(":", "_")
with open('../../data/germline/dataframes/predictorDF_{t}_1.txt'.format(t=timestamp), 'w') as f:                                                    # opening line to write to a file 
    
    header = "Chromosome"+ "\t"+"site"+"\t" +"triplet"+"\t"+"mutation_status"                        # creating the begining of the header 
    for trackname in tracksColFile_dict.keys():                                                     # the rest of the header is a function of tracks 
        for distance in list_of_surrounding_contexts:                                                # and distance (need a col for every track and for every distance value within ) 
            header = header + "\t"+str(trackname)+"-"+str(distance)
    for distance in list_of_surrounding_contexts:                                                    # creating the end of the header assoc with no track (the seqeunce at different 
        header = header + "\t"+"Apercent-"+str(distance)+ "\t"+"Gpercent-"+str(distance)+ "\t"+"Cpercent-"+str(distance)+ "\t"+"Tpercent-"+str(distance)   # distace values) 
    header = header +"\n"                                                                            # obviously needs to end with a \n 
    f.write(header)                                                                                  #write header to file 

    
    for site in tqdm(sites):
        if site[1] <= distance_max or site[1]+distance_max >= fastas_dict[site[0]][1]:               # only use sites that will have values for site +- max distance (buffer). second element in fastas dict is the length 
            list_sites_outOfBufferRange.append(site)
        
        else: 
            row = []
            row.extend([site[0], site[1]])
            alignment = fastas_dict[site[0]][0]                                                       #create the alingment from the list of fastas 
            
            
            #makes the triplet depending on itf mutant or nah 
            if site[2] ==1:                                     # 1 means mut==yes, muts_by-chrom_dict is literally a dictionary containing list of sites that are mutations in that chrom. 
                mutation_row = mutations_df[(mutations_df.chrom == site[0]) & (mutations_df.pos == site[1])]  #get the row containing mut info out of the df 

                mutation_bp = mutation_row.alt.values[0]
                mutated_triplet = str(alignment[0,site[1]-2]).upper()+str(mutation_bp)+str(alignment[0,site[1]]).upper() #these bounds ahve been tested 

                old_bp = mutation_row.ref.values[0]
                old_triplet = str(alignment[0,site[1]-2]).upper()+str(old_bp)+str(alignment[0,site[1]]).upper() #these bounds have been tested 

                row.extend([old_triplet, 1])

                seq_triplet = str(alignment[0,site[1]-2:site[1]+1].seq)#these bounds have been tested 
                if old_triplet.upper() != seq_triplet.upper(): 
                    alignment_discordant.append(old_triplet+" "+seq_triplet+" "+" "+str(mutation_row)+" "+str(sites.index(site))+" "+str(site))
                    
            else: 
                triplet= str(alignment[0,site[1]-2:site[1]+1].seq).upper()
                row.extend([triplet,0])
            
            #converting fro 0 to nomral chrom lists so can get correct files 
#             index_chromConversion = list_chroms.index(site[0])                                        #takes the first element of site (chr from tabix) and searches for the index within the list. 
#             chrom_forFiles = list_0chroms[index_chromConversion]                                      #used the previous index to get out the ch 0 number needed for the files : chr01 

            #track stuff 
            for trackname,track_val in tracksColFile_dict.items():                 
                
                 
                data_col = track_val[0] 
                global_or_tissue_specific = track_val[1]
                Na_is_0_or_NA = track_val[2]
                filename = track_val[3] 
                    
                for distance in list_of_surrounding_contexts: 
                    #track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]
                    if not [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]:                #if no value at that site 
                        if Na_is_0_or_NA == "Na=0": 
                            row.append(0)
                        else: 
                            row.extend(["NA"])                                                                                      

                    else:                                                   
                        track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]
                        multiple_values = []
                        if data_col == 4:
                            for element in track_output: 
                                multiple_values.append(float(element.split()[4]))
                            #print(multiple_values)
                            average_value = sum(multiple_values)/len(multiple_values) 
                            row.append(average_value)
                        elif data_col == 3: 
                            for element in track_output: 
                                multiple_values.append(float(element.split()[3]))
                            #print(multiple_values)
                            average_value = sum(multiple_values)/len(multiple_values) 
                            row.append(average_value)
                        elif data_col== 'binary': 
                            row.append(len(track_output))
                        else: 
                            print("ERROR: track coloumns not 4 or 3 or binary")
                            print(tracksColFile_dict[trackname])

            #sequence stuff                
            for distance in list_of_surrounding_contexts: 
                seq_around = str(alignment[0,site[1]-2:site[1]+1].seq)
                if seq_around != '': 
                    seq_around = str(alignment[0,site[1]-distance-1:site[1]+distance].seq)  #chnaged from  str(alignment[0,site[1]-distance:site[1]+distance+1].seq)
                    Acount = seq_around.count('a')+seq_around.count("A")
                    Gcount = seq_around.count('g')+seq_around.count("G")
                    Ccount = seq_around.count('c')+seq_around.count("C")
                    Tcount = seq_around.count('t')+seq_around.count("T")
                    Apercent = Acount/len(seq_around)
                    Gpercent = Gcount/len(seq_around)
                    Cpercent = Ccount/len(seq_around)
                    Tpercent = Tcount/len(seq_around)
                    row.extend([Apercent, Gpercent, Cpercent, Tpercent])
                else: 
                    row.extend(['NA','NA','NA','NA'])
                    list_no_seq_at_site.append(site)
            #print(header
            #print(row)          
            #print(len(row))
            row_string = str()
            for i in range(0,len(row)): 
                row_string = row_string+str(row[i])+"\t"
            row_string = row_string.rstrip("\t")
            row_string = row_string+"\n"
            f.write(row_string)
print(len(list_sites_outOfBufferRange))

  0%|                                                                                                           | 4/52471 [00:04<17:26:03,  1.20s/it]

## ** 1.7 printing error output of the model ** 

In [194]:
print(len(list_sites_outOfBufferRange))
print(len(alignment_discordant))
print(len( list_no_seq_at_site))

9
59
0


In [198]:
alignment_discordant[0].split("\t")

["GGGTAAAG GGG        chrom        pos   fake_end     ref alt  Fathers_age_at_conception  \\\n64329  chr3  104363463  104363464  GGTAAA   G                         22   \n\n       Mothers_age_at_conception  \n64329                         22   4 ['chr3', 104363463, 1]"]

In [200]:
for line in alignment_discordant: 
    if len(line.split()[0]) == 3: 
        print(line)

CAC ctc         chrom       pos  fake_end ref alt  Fathers_age_at_conception  \
15648  chr11  51391842  51391843   A   G                         58   

       Mothers_age_at_conception  
15648                         37   1350 ['chr11', 51391842, 1]
ACG AGG       chrom        pos   fake_end ref alt  Fathers_age_at_conception  \
4611  chr1  120696615  120696616   C   G                         32   

      Mothers_age_at_conception  
4611                         30   1527 ['chr1', 120696615, 1]


# **parallelizing**

In [17]:
from multiprocessing import Process 
from multiprocessing import Pool

In [18]:
def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

[1, 4, 9]


In [33]:
def f(site):
    if site[1] <= distance_max or site[1]+distance_max >= fastas_dict[site[0]][1]:               # only use sites that will have values for site +- max distance (buffer). second element in fastas dict is the length 
            list_sites_outOfBufferRange.append(site)
        
    else: 
        row = []
        row.extend([site[0], site[1]])
        alignment = fastas_dict[site[0]][0]                                                       #create the alingment from the list of fastas 


        #makes the triplet depending on itf mutant or nah 
        if site[2] ==1:                                     # 1 means mut==yes, muts_by-chrom_dict is literally a dictionary containing list of sites that are mutations in that chrom. 
            mutation_row = mutations_df[(mutations_df.chrom == site[0]) & (mutations_df.pos == site[1])]  #get the row containing mut info out of the df 

            mutation_bp = mutation_row.alt.values[0]
            mutated_triplet = str(alignment[0,site[1]-2]).upper()+str(mutation_bp)+str(alignment[0,site[1]]).upper() #these bounds ahve been tested 

            old_bp = mutation_row.ref.values[0]
            old_triplet = str(alignment[0,site[1]-2]).upper()+str(old_bp)+str(alignment[0,site[1]]).upper() #these bounds have been tested 

            row.extend([old_triplet, 1])

            seq_triplet = str(alignment[0,site[1]-2:site[1]+1].seq)#these bounds have been tested 
            if old_triplet.upper() != seq_triplet.upper(): 
                alignment_discordant.append(old_triplet+" "+seq_triplet+" "+" "+str(mutation_row)+" "+str(sites.index(site))+" "+str(site))

        else: 
            triplet= str(alignment[0,site[1]-2:site[1]+1].seq).upper()
            row.extend([triplet,0])

        #converting fro 0 to nomral chrom lists so can get correct files 
#             index_chromConversion = list_chroms.index(site[0])                                        #takes the first element of site (chr from tabix) and searches for the index within the list. 
#             chrom_forFiles = list_0chroms[index_chromConversion]                                      #used the previous index to get out the ch 0 number needed for the files : chr01 

        #track stuff 
        for trackname,track_val in tracksColFile_dict.items():                 


            data_col = track_val[0] 
            global_or_tissue_specific = track_val[1]
            Na_is_0_or_NA = track_val[2]
            filename = track_val[3] 

            for distance in list_of_surrounding_contexts: 
                #track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]
                if not [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]:                #if no value at that site 
                    if Na_is_0_or_NA == "Na=0": 
                        row.append(0)
                    else: 
                        row.extend(["NA"])                                                                                      

                else:                                                   
                    track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]
                    multiple_values = []
                    if data_col == 4:
                        for element in track_output: 
                            multiple_values.append(float(element.split()[4]))
                        #print(multiple_values)
                        average_value = sum(multiple_values)/len(multiple_values) 
                        row.append(average_value)
                    elif data_col == 3: 
                        for element in track_output: 
                            multiple_values.append(float(element.split()[3]))
                        #print(multiple_values)
                        average_value = sum(multiple_values)/len(multiple_values) 
                        row.append(average_value)
                    elif data_col== 'binary': 
                        row.append(len(track_output))
                    else: 
                        print("ERROR: track coloumns not 4 or 3 or binary")
                        print(tracksColFile_dict[trackname])

        #sequence stuff                
        for distance in list_of_surrounding_contexts: 
            seq_around = str(alignment[0,site[1]-2:site[1]+1].seq)
            if seq_around != '': 
                seq_around = str(alignment[0,site[1]-distance-1:site[1]+distance].seq)  #chnaged from  str(alignment[0,site[1]-distance:site[1]+distance+1].seq)
                Acount = seq_around.count('a')+seq_around.count("A")
                Gcount = seq_around.count('g')+seq_around.count("G")
                Ccount = seq_around.count('c')+seq_around.count("C")
                Tcount = seq_around.count('t')+seq_around.count("T")
                Apercent = Acount/len(seq_around)
                Gpercent = Gcount/len(seq_around)
                Cpercent = Ccount/len(seq_around)
                Tpercent = Tcount/len(seq_around)
                row.extend([Apercent, Gpercent, Cpercent, Tpercent])
            else: 
                row.extend(['NA','NA','NA','NA'])
                list_no_seq_at_site.append(site)
        #print(header
        #print(row)          
        #print(len(row))
        row_string = str()
        for i in range(0,len(row)): 
            row_string = row_string+str(row[i])+"\t"
        row_string = row_string.rstrip("\t")
        row_string = row_string+"\n"
        return row_string



In [25]:
sites[0]

['chr4', 147907388, 1]

In [37]:
f(sites[2])

'chr21\t1417507\tNNN\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\tNA\tNA\tNA\t0\t0\t0\t0\t0\t0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\n'

In [36]:
for site in sites[0:10]:
    print(f(site))

chr4	147907388	AAA	1	0.0	0.0	0.5417341228070175	0.46466	0.565276875	0.6800353357531765	0.0	0.0	0.9541180092592614	1.0526	1.0904975	0.6217522914521113	0.57553	0.38230666666666663	0.8276204593639565	0.7622	0.7014627272727272	0.7011680098280098	0.0	0.0164408	0.04055397066666665	0.0387503	0.0387503	0.06004818473282444	0	0	0	0	0	0.013109999999999998	0	0	1.1652631578947368	1.50124	1.50124	1.50124	0	0	2	21.0	21.0	21.5	1.0	0.0	0.0	0.0	0.35323383084577115	0.22885572139303484	0.10945273631840796	0.30845771144278605	0.3023348832558372	0.18104094795260237	0.17704114794260287	0.33958302084895753

chr8	37280633	TGT	1	1.52342	1.4206990909090906	0.826685210810809	0.24245	0.38473428571428575	0.7916427525622233	1.07802	1.0274371428571432	1.3322837939914152	0.41188	0.656560625	1.0021615578947365	0.34284	0.30701222222222224	0.6660858968347008	0.79536	0.5990331249999999	0.6122181364392679	0.0986448	0.0904243	0.06181737599999985	0.0387503	0.0387503	0.10800925895522388	0	0	0	0	0	0.01311	0	0.287	0.02825925925

In [34]:
# def f(x):
#     return x 

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [sites[0:10]]))

TypeError: '<=' not supported between instances of 'list' and 'int'